### 小売店の売り上げ履歴、顧客データからデータ分析してみる

In [2]:
import pandas as pd

In [5]:
uriage = pd.read_csv('uriage.csv')
kokyaku_data = pd.read_excel('kokyaku_daicho.xlsx')

In [6]:
uriage.head()

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美
1,2019-07-13 13:05:29,商 品 S,NaN,浅田賢二
2,2019-05-11 19:42:07,商 品 a,NaN,南部慶二
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒
4,2019-04-22 03:09:35,商品a,NaN,平田鉄二


In [7]:
kokyaku_data.head()

,顧客名,かな,地域,メールアドレス,登録日
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018/01/04
1,岡田 敏也,おかだ としや,E市,okada_toshiya@example.com,42782
2,芳賀 希,はが のぞみ,A市,haga_nozomi@example.com,2018/01/07
3,荻野 愛,おぎの あい,F市,ogino_ai@example.com,42872
4,栗田 憲一,くりた けんいち,E市,kurita_kenichi@example.com,43127


In [8]:
#データの揺れを確認
#商品はアルファベット26個分存在する

uriage['item_name'].head()

0      商品A
1    商 品 S
2    商 品 a
3      商品Z
4      商品a
Name: item_name, dtype: object

In [12]:
len(uriage['item_name'].value_counts())

99

In [14]:
#a と A で別の商品として扱われているため全て大文字に揃えスペースを削除

uriage['item_name'] =uriage['item_name'].str.upper()
uriage['item_name'] =uriage['item_name'].str.replace(' ', '')
uriage['item_name'] =uriage['item_name'].str.replace('　', '')

In [19]:
print(uriage['item_name'].unique())
print(len(uriage['item_name'].value_counts()))

['商品A' '商品S' '商品Z' '商品V' '商品O' '商品U' '商品L' '商品C' '商品I' '商品R' '商品X' '商品G'
 '商品P' '商品Q' '商品Y' '商品N' '商品W' '商品E' '商品K' '商品B' '商品F' '商品D' '商品M' '商品H'
 '商品T' '商品J']
26


In [62]:
#購入月データ

uriage['purchase_date'] = pd.to_datetime(uriage['purchase_date'])
uriage['purchase_month'] = uriage['purchase_date'].dt.strftime('%Y%m')

In [63]:
uriage.head()

,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美,201906
1,2019-07-13 13:05:29,商品S,1900.0,浅田賢二,201907
2,2019-05-11 19:42:07,商品A,100.0,南部慶二,201905
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒,201902
4,2019-04-22 03:09:35,商品A,100.0,平田鉄二,201904


In [20]:
#欠損値

uriage.isnull().sum()

purchase_date      0
item_name          0
item_price       387
customer_name      0
dtype: int64

In [29]:
#item_price の欠損値は同商品の金額から補完する

flg_is_null = uriage['item_price'].isnull()

for trg in list(uriage.loc[flg_is_null, 'item_name'].unique()):
    price = uriage.loc[(~flg_is_null) & (uriage['item_name'] == trg), 'item_price'].max()
    uriage['item_price'].loc[(flg_is_null) & (uriage['item_name'] == trg)] = price
    

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [30]:
uriage.head()

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02:34,商品A,100.0,深井菜々美
1,2019-07-13 13:05:29,商品S,1900.0,浅田賢二
2,2019-05-11 19:42:07,商品A,100.0,南部慶二
3,2019-02-12 23:40:45,商品Z,2600.0,麻生莉緒
4,2019-04-22 03:09:35,商品A,100.0,平田鉄二


In [31]:
#kokyaku_dataのデータの揺れ

kokyaku_data.head()

,顧客名,かな,地域,メールアドレス,登録日
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018/01/04
1,岡田 敏也,おかだ としや,E市,okada_toshiya@example.com,42782
2,芳賀 希,はが のぞみ,A市,haga_nozomi@example.com,2018/01/07
3,荻野 愛,おぎの あい,F市,ogino_ai@example.com,42872
4,栗田 憲一,くりた けんいち,E市,kurita_kenichi@example.com,43127


In [32]:
kokyaku_data['顧客名'] = kokyaku_data['顧客名'].str.upper()
kokyaku_data['顧客名'] = kokyaku_data['顧客名'].str.replace(' ', '')
kokyaku_data['顧客名'] = kokyaku_data['顧客名'].str.replace('　', '')

In [33]:
kokyaku_data.head()

,顧客名,かな,地域,メールアドレス,登録日
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018/01/04
1,岡田敏也,おかだ としや,E市,okada_toshiya@example.com,42782
2,芳賀希,はが のぞみ,A市,haga_nozomi@example.com,2018/01/07
3,荻野愛,おぎの あい,F市,ogino_ai@example.com,42872
4,栗田憲一,くりた けんいち,E市,kurita_kenichi@example.com,43127


In [36]:
#登録日 の揺れ

flg_is_serial = kokyaku_data['登録日'].astype('str').str.isdigit()

In [ ]:
fromSerial = pd.to_timedelta(kokyaku_data.loc[flg_is_serial, '登録日'].astype('float'), unit='D') + pd.to_datetime('1900/01/01')

In [47]:
fromSerial

1     2017-02-18
3     2017-05-19
4     2018-01-29
21    2017-07-06
27    2017-06-17
47    2017-01-08
49    2017-07-15
53    2017-04-10
76    2018-03-31
80    2018-01-12
99    2017-06-01
114   2018-06-05
118   2018-01-31
122   2018-04-18
139   2017-05-27
143   2017-03-26
155   2017-01-21
172   2018-03-24
179   2017-01-10
183   2017-07-26
186   2018-07-15
192   2018-06-10
Name: 登録日, dtype: datetime64[ns]

In [41]:
fromString = pd.to_datetime(kokyaku_data.loc[~flg_is_serial, '登録日'])
fromString

0     2018-01-04
2     2018-01-07
5     2017-06-20
6     2018-06-11
7     2017-05-19
         ...    
195   2017-06-20
196   2018-06-20
197   2017-04-29
198   2019-04-19
199   2019-04-23
Name: 登録日, Length: 178, dtype: datetime64[ns]

In [44]:
kokyaku_data['登録日'] = pd.concat([fromSerial, fromString])
kokyaku_data.head()

,顧客名,かな,地域,メールアドレス,登録日
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018-01-04
1,岡田敏也,おかだ としや,E市,okada_toshiya@example.com,2017-02-18
2,芳賀希,はが のぞみ,A市,haga_nozomi@example.com,2018-01-07
3,荻野愛,おぎの あい,F市,ogino_ai@example.com,2017-05-19
4,栗田憲一,くりた けんいち,E市,kurita_kenichi@example.com,2018-01-29


In [48]:
#登録日から月を算出

kokyaku_data['登録年月'] = kokyaku_data['登録日'].dt.strftime('%Y%m')

In [49]:
kokyaku_data.head()

,顧客名,かな,地域,メールアドレス,登録日,登録年月
0,須賀ひとみ,すが ひとみ,H市,suga_hitomi@example.com,2018-01-04,201801
1,岡田敏也,おかだ としや,E市,okada_toshiya@example.com,2017-02-18,201702
2,芳賀希,はが のぞみ,A市,haga_nozomi@example.com,2018-01-07,201801
3,荻野愛,おぎの あい,F市,ogino_ai@example.com,2017-05-19,201705
4,栗田憲一,くりた けんいち,E市,kurita_kenichi@example.com,2018-01-29,201801


In [64]:
join_data = pd.merge(uriage, kokyaku_data, left_on='customer_name', right_on='顧客名', how='left')

In [65]:
join_data = join_data.drop('customer_name', axis=1)
join_data.head()

,purchase_date,item_name,item_price,purchase_month,顧客名,かな,地域,メールアドレス,登録日,登録年月
0,2019-06-13 18:02:34,商品A,100.0,201906,深井菜々美,ふかい ななみ,C市,fukai_nanami@example.com,2017-01-26,201701
1,2019-07-13 13:05:29,商品S,1900.0,201907,浅田賢二,あさだ けんじ,C市,asada_kenji@example.com,2018-04-07,201804
2,2019-05-11 19:42:07,商品A,100.0,201905,南部慶二,なんぶ けいじ,A市,nannbu_keiji@example.com,2018-06-19,201806
3,2019-02-12 23:40:45,商品Z,2600.0,201902,麻生莉緒,あそう りお,D市,asou_rio@example.com,2018-07-22,201807
4,2019-04-22 03:09:35,商品A,100.0,201904,平田鉄二,ひらた てつじ,D市,hirata_tetsuji@example.com,2017-06-07,201706


In [67]:
dump_data = join_data[['purchase_date', 'purchase_month', 'item_name','item_price','顧客名','かな','地域','メールアドレス','登録日','登録年月']]

In [68]:
dump_data.to_csv('output_csv/dump_data.csv', index=False)

In [73]:
import_data = pd.read_csv('output_csv/dump_data.csv')
import_data.head()

,purchase_date,purchase_month,item_name,item_price,顧客名,かな,地域,メールアドレス,登録日,登録年月
0,2019-06-13 18:02:34,201906,商品A,100.0,深井菜々美,ふかい ななみ,C市,fukai_nanami@example.com,2017-01-26 00:00:00,201701
1,2019-07-13 13:05:29,201907,商品S,1900.0,浅田賢二,あさだ けんじ,C市,asada_kenji@example.com,2018-04-07 00:00:00,201804
2,2019-05-11 19:42:07,201905,商品A,100.0,南部慶二,なんぶ けいじ,A市,nannbu_keiji@example.com,2018-06-19 00:00:00,201806
3,2019-02-12 23:40:45,201902,商品Z,2600.0,麻生莉緒,あそう りお,D市,asou_rio@example.com,2018-07-22 00:00:00,201807
4,2019-04-22 03:09:35,201904,商品A,100.0,平田鉄二,ひらた てつじ,D市,hirata_tetsuji@example.com,2017-06-07 00:00:00,201706


In [75]:
#データ集計
#購入月と商品名
byItem = import_data.pivot_table(index='purchase_month', columns='item_name', aggfunc='size', fill_value=0)
byItem

item_name,商品A,商品B,商品C,商品D,商品E,商品F,商品G,商品H,商品I,商品J,...,商品Q,商品R,商品S,商品T,商品U,商品V,商品W,商品X,商品Y,商品Z
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,18,13,19,17,18,15,11,16,18,17,...,17,21,20,17,7,22,13,14,10,0
201902,19,14,26,21,16,14,14,17,12,14,...,22,22,22,23,19,22,24,16,11,1
201903,17,21,20,17,9,27,14,18,12,16,...,23,16,20,12,23,18,16,21,16,0
201904,17,19,24,20,18,17,14,11,18,13,...,20,20,16,16,11,15,14,16,20,0
201905,24,14,16,14,19,18,23,15,16,11,...,13,22,18,16,16,9,21,16,20,0
201906,24,12,11,19,13,18,15,13,19,22,...,15,16,21,12,18,20,17,15,13,0
201907,20,20,17,17,12,17,19,19,19,23,...,15,19,23,21,13,28,16,18,12,0


In [77]:
#購入月と商品売り上げ
byPrice = import_data.pivot_table(index='purchase_month', columns='item_name' ,values='item_price', aggfunc='sum', fill_value=0)
byPrice

item_name,商品A,商品B,商品C,商品D,商品E,商品F,商品G,商品H,商品I,商品J,...,商品Q,商品R,商品S,商品T,商品U,商品V,商品W,商品X,商品Y,商品Z
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,1800,2600,5700,6800,9000,9000,7700,12800,16200,17000,...,28900,37800,38000,34000,14700,48400,29900,33600,25000,0
201902,1900,2800,7800,8400,8000,8400,9800,13600,10800,14000,...,37400,39600,41800,46000,39900,48400,55200,38400,27500,2600
201903,1700,4200,6000,6800,4500,16200,9800,14400,10800,16000,...,39100,28800,38000,24000,48300,39600,36800,50400,40000,0
201904,1700,3800,7200,8000,9000,10200,9800,8800,16200,13000,...,34000,36000,30400,32000,23100,33000,32200,38400,50000,0
201905,2400,2800,4800,5600,9500,10800,16100,12000,14400,11000,...,22100,39600,34200,32000,33600,19800,48300,38400,50000,0
201906,2400,2400,3300,7600,6500,10800,10500,10400,17100,22000,...,25500,28800,39900,24000,37800,44000,39100,36000,32500,0
201907,2000,4000,5100,6800,6000,10200,13300,15200,17100,23000,...,25500,34200,43700,42000,27300,61600,36800,43200,30000,0


In [78]:
#購入月と各顧客の購入数
byCustomer = import_data.pivot_table(index='purchase_month', columns='顧客名', aggfunc='size', fill_value=0)
byCustomer

顧客名,さだ千佳子,中仁晶,中田美智子,丸山光臣,久保田倫子,亀井一徳,五十嵐春樹,井上桃子,井口寛治,井川真悠子,...,香椎優一,高原充則,高梨結衣,高沢美咲,高田さんま,鳥居広司,鶴岡薫,麻生莉緒,黄川田博之,黒谷長利
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,3,1,4,2,2,0,5,3,3,1,...,0,1,1,1,5,2,0,2,2,5
201902,9,1,2,2,1,4,2,1,0,4,...,4,0,3,2,0,1,2,4,0,1
201903,1,2,1,6,1,4,3,3,2,2,...,3,1,6,2,4,2,4,2,2,1
201904,0,3,1,2,0,2,2,0,3,2,...,2,4,2,3,4,3,2,1,2,0
201905,3,2,5,2,4,1,2,1,3,3,...,1,1,1,0,2,2,3,4,4,1
201906,1,3,0,4,1,1,1,2,2,3,...,7,3,0,2,1,0,2,1,2,4
201907,3,0,3,2,5,3,5,2,5,5,...,2,4,4,2,0,2,4,3,4,1


In [82]:
#集計期間で購入していないユーザー

away_data = pd.merge(uriage, kokyaku_data, left_on='customer_name', right_on='顧客名', how='right')

In [83]:
away_data[away_data['purchase_date'].isnull()][['顧客名', 'メールアドレス', '登録日']]

,顧客名,メールアドレス,登録日
2999,福井美希,fukui_miki1@example.com,2019-04-23
